In [1]:
import os
os.environ["SPARK_HOME"] = "/home/osboxes/spark/spark-2.4.0-bin-hadoop2.7/"

import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import levenshtein 
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
#conf = pyspark.SparkConf().setAppName('appName').setMaster('spark://192.168.11.128:8080')

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

#spark.enableHiveSupport()

In [6]:
from pyspark.sql.types import *
simpleSchema = [StructField("Position",IntegerType(),False),
                StructField("Track_Name",StringType(),False),
                StructField("Artist",StringType(),False),
                StructField("Streams",FloatType(),True),                
                StructField("Date", StringType(),False),
                StructField("region", StringType(), False),
                StructField("spotify_id",StringType(),False),
               ]

simple_struct = StructType(fields=simpleSchema)
simple_struct




StructType(List(StructField(Position,IntegerType,false),StructField(Track_Name,StringType,false),StructField(Artist,StringType,false),StructField(Streams,FloatType,true),StructField(Date,StringType,false),StructField(region,StringType,false),StructField(spotify_id,StringType,false)))

In [ ]:
def set_Schema(cols):
    simpleSchema = []
    
    for i in cols: 
        simpleSchema.append(StructField(i[0],i[1],i[2])) 
        
    return StructType(fields=simpleSchema)

In [ ]:
columns = [('Position',IntegerType(), False ),
           ('Track Name',StringType(), False), 
           ('Artist', StringType(), False), 
           ('Streams', FloatType(), False), 
           ('Date', StringType(), False), 
           ('region', StringType(), False), 
           ('spotify_id', StringType(), True)]
simple_struct = set_Schema(columns)
simple_struct

## Reading ftom postgres tables

In [2]:

jdbcDF2 = spark.read \
    .jdbc("jdbc:postgresql://osboxes:5432/postgres", "spotify_top_200_weekly", \
          properties={"user": "postgres", "password": "postgres"})

In [28]:
jdbcDF2.count()
jdbcDF2.filter(jdbcDF2.spotify_id=='5mZXWEH2eh8zMZGCxT5aW0').show()


+--------+--------------------+------+-------+--------------------+------+--------------------+
|Position|          Track Name|Artist|Streams|                date|region|          spotify_id|
+--------+--------------------+------+-------+--------------------+------+--------------------+
|      47|Yah Yah (feat. Ro...|Eminem| 968088|2020-01-17--2020-...|    gb|5mZXWEH2eh8zMZGCx...|
|      83|Yah Yah (feat. Ro...|Eminem|8695060|2020-01-17--2020-...|global|5mZXWEH2eh8zMZGCx...|
|     136|Yah Yah (feat. Ro...|Eminem| 485054|2020-01-24--2020-...|    gb|5mZXWEH2eh8zMZGCx...|
+--------+--------------------+------+-------+--------------------+------+--------------------+



## Option to add to spark.read.csv to ensure string in quotes and commas are treated as one

In [31]:
from pyspark.sql.functions import col, split
#df_weeklyTop200 = spark.read.csv('../datasets/spotify_top_200_weekly.csv', header=True, nullValue='', schema=simple_struct)
df_weeklyTop200 = spark.read.csv('../datasets/spotify_top_200_weekly.csv',header=True,encoding="UTF-8", quote='"',escape='"') 
                                            
'''                                             
multiLine=True, 
ignoreLeadingWhiteSpace=True, 
ignoreTrailingWhiteSpace=True, 
encoding="UTF-8",
sep=',',                                            
maxColumns=2,inferSchema=True)
'''

df_weeklyTop200 = df_weeklyTop200.withColumnRenamed("Track Name","Track_Name")
df_weeklyTop200 = df_weeklyTop200.withColumn("start_date", split(col("date"), "--").getItem(0)).withColumn("end_date", split(col("date"), "--").getItem(1))
df_weeklyTop200.show(2, truncate=False, vertical =True)

-RECORD 0----------------------------------------------
 Position   | 1                                        
 Track_Name | Someone You Loved                        
 Artist     | Lewis Capaldi                            
 Streams    | 3338234                                  
 date       | 2019-03-01--2019-03-08                   
 region     | gb                                       
 spotify_id | 2TIlqbIneP0ZY1O0EzYLlc                   
 start_date | 2019-03-01                               
 end_date   | 2019-03-08                               
-RECORD 1----------------------------------------------
 Position   | 2                                        
 Track_Name | break up with your girlfriend, i'm bored 
 Artist     | Ariana Grande                            
 Streams    | 3041558                                  
 date       | 2019-03-01--2019-03-08                   
 region     | gb                                       
 spotify_id | 4kV4N9D1iKVxx1KLvtTpjS            

In [ ]:
df_weeklyTop200.printSchema()

In [32]:
df_weeklyTop200.describe().show(vertical=True)

-RECORD 0--------------------------
 summary    | count                
 Position   | 20800                
 Track_Name | 20800                
 Artist     | 20800                
 Streams    | 20800                
 date       | 20800                
 region     | 20800                
 spotify_id | 20800                
 start_date | 20800                
 end_date   | 20800                
-RECORD 1--------------------------
 summary    | mean                 
 Position   | 100.5                
 Track_Name | 1619.2583333333334   
 Artist     | null                 
 Streams    | 4779709.056682692    
 date       | null                 
 region     | null                 
 spotify_id | null                 
 start_date | null                 
 end_date   | null                 
-RECORD 2--------------------------
 summary    | stddev               
 Position   | 57.735693120535515   
 Track_Name | 752.4803188004807    
 Artist     | null                 
 Streams    | 5885641.043877

In [ ]:
df_weeklyTop200.show()

In [30]:
df_weeklyTop200 = df_weeklyTop200.withColumn("start_date", df_weeklyTop200["start_date"].cast(DateType()))
df_weeklyTop200 = df_weeklyTop200.withColumn("end_date", df_weeklyTop200["end_date"].cast(DateType()))
df_weeklyTop200.printSchema()


root
 |-- Position: string (nullable = true)
 |-- Track_Name: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Streams: string (nullable = true)
 |-- date: string (nullable = true)
 |-- region: string (nullable = true)
 |-- spotify_id: string (nullable = true)
 |-- start_date: date (nullable = true)
 |-- end_date: date (nullable = true)



In [27]:
df_weeklyTop200.filter(df_weeklyTop200.spotify_id == '5mZXWEH2eh8zMZGCxT5aW0').show()

+--------+--------------------+------+-------+--------------------+------+--------------------+----------+----------+
|Position|          Track_Name|Artist|Streams|                date|region|          spotify_id|start_date|  end_date|
+--------+--------------------+------+-------+--------------------+------+--------------------+----------+----------+
|      47|Yah Yah (feat. Ro...|Eminem| 968088|2020-01-17--2020-...|    gb|5mZXWEH2eh8zMZGCx...|2020-01-17|2020-01-24|
|      83|Yah Yah (feat. Ro...|Eminem|8695060|2020-01-17--2020-...|global|5mZXWEH2eh8zMZGCx...|2020-01-17|2020-01-24|
|     136|Yah Yah (feat. Ro...|Eminem| 485054|2020-01-24--2020-...|    gb|5mZXWEH2eh8zMZGCx...|2020-01-24|2020-01-31|
+--------+--------------------+------+-------+--------------------+------+--------------------+----------+----------+



In [ ]:
df_weeklyTop200.createOrReplaceTempView('df_weeklyTop200')
df_weeklyTop200.count()

In [ ]:
spark.sql("select * from df_weeklyTop200 where region <> 'global'and region <> 'gb'").show()
spark.sql("select * from df_weeklyTop200").count()

In [ ]:
spark.sql("select * from df_weeklyTop200 \
          where start_date = '2020-01-17' and end_date = '2020-01-24' \
          and position in (47,83)").show(vertical =True, truncate=False)



In [ ]:
df.write \
    .jdbc("jdbc:postgresql://osboxes:5432/postgres", "spotify_top_200_weekly",
          properties={"user": "postgres", "password": "postgres"})